<a href="https://colab.research.google.com/github/JossiasZ/ML-models/blob/main/Artificial_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

dados = pd.read_csv("system1data.csv")
dados.head()
print(dados)

# Defining features and labels
X = ['Voc/MaxVoc', 'Isc/MaxIsc', 'G/1000', 'AT/50']
caracteristicas = dados[X]
y = dados.Fault

# Transforming the labels into the desired format [1,0,0]
y_transformed = to_categorical(y)
y_numeric = y_transformed.astype(int)

X_scaled = caracteristicas

from sklearn.model_selection import train_test_split
X_trein, X_teste, y_trein, y_teste = train_test_split(X_scaled, y_numeric, test_size=0.2, random_state=0)

modelo = Sequential()

modelo.add(Dense(units=4, input_shape=(4,)))
modelo.add(Dense(units=74, activation='relu'))
modelo.add(Dense(units=78, activation='relu'))
modelo.add(Dense(units=91, activation='relu'))
modelo.add(Dense(units=3, activation='softmax'))

learning_rate = 0.0062
optimizer = Adam(learning_rate=learning_rate)

modelo.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

modelo.summary()

history = modelo.fit(x=X_trein, y=y_trein, epochs=1000, validation_data=(X_teste, y_teste), verbose=1)

modelo.evaluate(X_scaled, y_numeric)

desempenho_do_modelo = pd.DataFrame(history.history)

modelo.save('modelo_treinado.RNA')

# Predictions for the training data
y_pred_trein = np.argmax(modelo.predict(X_trein), axis=-1)

# Predictions for the test data
y_pred_teste = np.argmax(modelo.predict(X_teste), axis=-1)

# Confusion matrix for the training data
cm_trein = confusion_matrix(np.argmax(y_trein, axis=-1), y_pred_trein)
print("Matriz de Confusão (Treinamento):\n", cm_trein)

# Confusion matrix for the test data
cm_teste = confusion_matrix(np.argmax(y_teste, axis=-1), y_pred_teste)
print("Matriz de Confusão (Teste):\n", cm_teste)

# Classification report for the test data
report_teste = classification_report(np.argmax(y_teste, axis=-1), y_pred_teste, output_dict=True)
print("Classification Report (Teste):")
for key, value in report_teste.items():
    if isinstance(value, dict):
        print(f"{key.capitalize()}:")
        for metric, score in value.items():
            if metric != 'support':
                print(f"    {metric.capitalize()}: {score:.4f}")
    else:
        print(f"{key.capitalize()}: {value:.4f}")

# Classification report for the training data
report_treinamento = classification_report(np.argmax(y_trein, axis=-1), y_pred_trein, output_dict=True)
print("Classification Report (Treinamento):")
for key, value in report_treinamento.items():
    if isinstance(value, dict):
        print(f"{key.capitalize()}:")
        for metric, score in value.items():
            if metric != 'support':
                print(f"    {metric.capitalize()}: {score:.4f}")
    else:
        print(f"{key.capitalize()}: {value:.4f}")

ax = desempenho_do_modelo[['loss', 'val_loss', 'accuracy', 'val_accuracy']].plot()
ax.set_xlabel('Época')

plt.show()

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
import numpy as np
from keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns

dados = pd.read_csv("system2data.csv")
dados.head()

# Loading the trained model
modelo = tf.keras.models.load_model('modelo_treinado.RNA')

# Defining attributes and labels
X = ['Voc/MaxVoc', 'Isc/MaxIsc', 'G/1000', 'AT/50']
caracteristicas = dados[X]

y = dados.Fault

# Transforming the labels into the desired format [1,0,0]
y_transformed = to_categorical(y)
y_numeric = y_transformed.astype(int)

X_scaled = caracteristicas

# Predictions on new data
previsoes = modelo.predict(X_scaled)

# Converting predictions into labels
rotulos_previstos = np.argmax(previsoes, axis=1)

# Calculating accuracy
acuracia = round(accuracy_score(np.argmax(y_numeric, axis=1), rotulos_previstos), 4)
print("Acurácia:", acuracia)

# Plotting the confusion matrix
matriz_confusao = confusion_matrix(np.argmax(y_numeric, axis=1), rotulos_previstos)
print("Matriz de Confusão:\n", matriz_confusao)

# Classification report
report = classification_report(np.argmax(y_numeric, axis=1), rotulos_previstos, digits=4)
print("Classification Report:\n", report)